In [1]:
import tifffile
import pandas

from griottes.analyse import cell_property_extraction
from griottes.graphmaker import graph_generation_func
from griottes.graphplotter import graph_plot
import griottes

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load image and mask of nuclei

From the image we can extract the single cell information. For the analysis, it is necessary that the channels be structured as $Z \times X \times Y \times C$, where $C$ is the fluorescence channels in a multi-channel image.

The file `total_image.tiff` can be downloaded from by executing the box below.

In [5]:
import urllib3
import shutil
import os

url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/3D_spheroid_multichannel_image.tif'
filename = '3D_spheroid_multichannel_image.tiff'

if not os.path.exists(filename):

    c = urllib3.PoolManager()

    with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)

    resp.release_conn()
    
assert os.path.exists(filename)

# Extract single-cell information from the image

In [9]:
spheroid_image = tifffile.imread('3D_spheroid_multichannel_image.tiff')

spheroid_image.shape

(4, 513, 512, 512)

Calculating geometrical properties


  0%|          | 0/57 [00:00<?, ?it/s]/Users/gustaveronteix/anaconda3/lib/python3.8/site-packages/griottes/analyse/cell_property_extraction.py:75: RuntimeWarning: divide by zero encountered in double_scalars
  properties.loc[ind, "eccentricity"] = np.abs(var[0]) / np.sqrt(
  4%|▎         | 2/57 [00:00<00:06,  8.65it/s]/Users/gustaveronteix/anaconda3/lib/python3.8/site-packages/griottes/analyse/cell_property_extraction.py:75: RuntimeWarning: divide by zero encountered in double_scalars
  properties.loc[ind, "eccentricity"] = np.abs(var[0]) / np.sqrt(
/Users/gustaveronteix/anaconda3/lib/python3.8/site-packages/griottes/analyse/cell_property_extraction.py:75: RuntimeWarning: divide by zero encountered in double_scalars
  properties.loc[ind, "eccentricity"] = np.abs(var[0]) / np.sqrt(
/Users/gustaveronteix/anaconda3/lib/python3.8/site-packages/griottes/analyse/cell_property_extraction.py:75: RuntimeWarning: divide by zero encountered in double_scalars
  properties.loc[ind, "eccentricity"] 

Done geometrical properties
Calculating voronoi


38it [00:00, 40.76it/s]


QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Q12 Qz Qc Qt Qbb
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 2145456857  delaunay  Q12-allow-wide  Qz-infinity-point
  Qcoplanar-keep  Qtriangulate  Qbbound-last  _pre-merge  _zero-centrum
  Qinterior-keep  Pgood  _max-width 59  Error-roundoff 1.3e-12
  _one-merge 1.2e-11  Visible-distance 7.8e-12  U-max-coplanar 7.8e-12
  Width-outside 1.6e-11  _wide-facet 4.7e-11  _maxoutside 1.6e-11

precision problems (corrected unless 'Q0' or an error)
      1 zero divisors during gaussian elimination

The input to qhull appears to be less than 4 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v5): 6.2e+02 2.4e+02 3.6e+02 2e+02
- p6(v4): 6.1e+02 2.4e+02 3.6e+02 6.5e+02
- p3(v3): 6.4e+02 2.4e+02 3.6e+02 3e+02
- p2(v2): 6.5e+02 2.4e+02 3.6e+02 3.5e+02
- p0(v1): 5.9e+02 2.4e+02 3.6e

In [ ]:
### THIS DOENS'T WORK - return an empty table. 
### TO BE FIXED!

print(total_image.shape)
spheroid_image = total_image[:, 200:400, 300:500,]

prop = cell_property_extraction.get_cell_properties(
    spheroid_image,
    mask_channel = 3,
    analyze_fluo_channels = True,
    fluo_channel_analysis_method = 'local_voronoi',
    cell_geometry_properties = True,
    radius = 35,
    labeled_voronoi_tesselation = False,
    percentile = 95,
    min_area = 50
    )

In [ ]:
table = pd.read_csv('3D_spheroid_cell_coordinates.csv')
table

From the extracted positions of the cell centers, it is possible to generate a network representation of the 3D spheroid. For practical purposes, we have saved the dataframe containing the previously extracted information which is re-used below.

For illustration purposes we decide to attribute different cell types depending on the fluorescence measured within the mask. Any cell which mean_intensity_1 is above the median is considered CD146+, otherwise it is CD146-.

In [ ]:
def get_cell_type(prop_table:pd.DataFrame):
    prop = prop_table.copy()
    prop['cell_type'] = (prop.mean_intensity_1 > prop.mean_intensity_1.median()).astype(int)
    prop.index = np.arange(len(prop))

    legend_list = ['CD146-', 'CD146+']
    prop['legend'] = [legend_list[prop.loc[i, 'cell_type']] for i in range(len(prop))]

    color_list = [plt.cm.Set3(i) for i in range(2)]
    prop['color'] = [color_list[prop.loc[i, 'cell_type']] for i in range(len(prop))]

    return prop

In [ ]:
prop = get_cell_type(table)

# Delaunay-based network construction

In [ ]:
descriptors = ['x', 'y', 'legend', 'color', 'cell_type']

G = graph_generation_func.generate_delaunay_graph(prop, 
                                                 descriptors = descriptors, 
                                                 distance = 100,
                                                 )

From the graph G it is possible to plot a visual representation of the network. Griottes contains several specific plotting functions adapted for the network representation of tisssues. These functions are called through the graph_plot module. Here we call a specific function for 3D plots.

In [ ]:
graph_plot.network_plot_3D(G, 
                figsize = (7, 7),
                alpha_line=0.6,
                scatterpoint_size=10,
                legend=True,
                legend_fontsize = 18,
                theta = -0,
                psi = -0,
                xlim = (prop.x.min() - 5, prop.x.max() + 5),
                ylim = (prop.y.min() - 5, prop.y.max() + 5),
                zlim = (prop.z.min() - 5, prop.z.max() + 5))